## 🔰 Phase 0: Setup & Environment

### `🔹 Install & Configure`

-    ✅ Install PostgreSQL (latest version)

In [ ]:
sudo apt install postgresql postgresql-contrib

- ✅ Enable PostgreSQL Service

In [ ]:
sudo systemctl start postgresql

🔹 Bonus Service Commands

In [ ]:
sudo systemctl enable postgresql   # Auto-start on boot
sudo systemctl stop postgresql     # Stop service
sudo systemctl restart postgresql  # Restart after config changes
sudo systemctl status postgresql   # Check status

🔹 Create Environment

- ✅ Create Role & Database

In [ ]:
CREATE ROLE hemendra WITH LOGIN PASSWORD 'yourpassword';
CREATE DATABASE dev_db OWNER hemendra;
GRANT ALL PRIVILEGES ON DATABASE dev_db TO hemendra;

1) `sudo apt install postgresql postgresql-contrib`


`sudo`: Runs the command with administrator privileges (superuser).

`apt install`: Installs software packages using Ubuntu’s package manager (apt).

`postgresql`: Installs the main PostgreSQL server (database engine).

`postgresql-contrib`: Installs extra useful tools and extensions that are officially supported by PostgreSQL (like pg_stat_statements, uuid-ossp, etc).

🧠 In short:

Installs PostgreSQL database + some official add-ons that help in real-world use cases (like performance tracking, UUID support, etc).

----------------------------


2) `sudo systemctl start postgresql`

`systemctl`: A tool to manage services (daemons) in Ubuntu.
`
`start postgresql`: Starts the PostgreSQL server as a background service (so it begins listening for connections on port 5432 by default).

🧠 In short:

Starts the PostgreSQL database service, so you can connect, create databases, and run queries.

-------------------


### `🔹 Create Your Environment`

-    ✅ Create a new user (role) and database

In [ ]:
CREATE ROLE hemendra WITH LOGIN PASSWORD 'yourpassword';
CREATE DATABASE dev_db OWNER hemendra;
GRANT ALL PRIVILEGES ON DATABASE dev_db TO hemendra;


## 🧱 Phase 1: PostgreSQL Table Management + CRUD Operations (DDL + DML Essentials)

### 🔹 1. Table Creation & Schema Management (DDL)

In [ ]:
CREATE TABLE employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    department VARCHAR(50),
    salary NUMERIC(10,2),
    joining_date DATE
);

ALTER TABLE employees ADD COLUMN email VARCHAR(100); 
ALTER TABLE employees DROP COLUMN email;
ALTER TABLE employees RENAME COLUMN department TO dept;
ALTER TABLE employees ALTER COLUMN salary TYPE INTEGER;
ALTER TABLE employees RENAME TO staff;
DROP TABLE staff;

CREATE SCHEMA company;
SET search_path TO company;


- ✅ CRUD Operations

    - `CREATE, INSERT, SELECT, UPDATE, DELETE`

    - Use `SERIAL / BIGSERIAL` for auto-incrementing PKs

    - RETURNING clause to fetch inserted/updated rows

--------------------------- 

## 🛡️ Phase 2: Normalization, Constraints & Data Types


- ✅ Normalization (1NF to BCNF)

   - Remove redundancy, avoid update anomalies

   - Split large unstructured tables into structured ones using:

      -  1NF: Atomic values

      -  2NF: Full functional dependency

      - 3NF: No transitive dependency

      -  BCNF: Even stricter version of 3NF

You can practice normalization with employee, department, and salary tables.



- ✅ Important PostgreSQL Constraints

   - NOT NULL, UNIQUE, PRIMARY KEY, FOREIGN KEY

   - CHECK for value validation

   - DEFAULT values for fields

   - composite keys, multi-column constraints

- ✅ Strong PostgreSQL Data Types

   - Category ----->	Types

   - Text ----->	`TEXT, VARCHAR(n)`

   - Numbers ----->	`INTEGER, BIGINT, NUMERIC, REAL, DECIMAL`

   - Dates/Times ----->	`DATE, TIMESTAMP WITH TIME ZONE`

   - Boolean ----->	`BOOLEAN`

   - Special ----->	`JSONB, UUID, ARRAY, BYTEA`




-----------------------


## 🔗 Phase 3: Joins, Subqueries & Grouping

- ✅ Master These SQL Patterns

    - JOINs: `INNER, LEFT, RIGHT, FULL, SELF JOIN`

    - Aggregate functions: `COUNT(), SUM(), AVG(), MAX(), MIN()`

    - Grouping & Filtering: `GROUP BY, HAVING, DISTINCT`

- ✅ Subqueries

    - Single-row and multi-row subqueries

    - Correlated subqueries (used inside WHERE/SELECT)

---------------

## ⚙️ Phase 4: Advanced Features of PostgreSQL

- ✅ Common Table Expressions (CTEs)

    Use for readability and modular queries

In [ ]:
WITH dept_count AS (
  SELECT department, COUNT(*) AS count FROM employees GROUP BY department
)
SELECT * FROM dept_count WHERE count > 5;


- ✅ Window Functions

    `RANK(), ROW_NUMBER(), DENSE_RANK(), LEAD(), LAG()`, running totals

In [ ]:
SELECT name, salary, RANK() OVER (ORDER BY salary DESC) FROM employees;

✅ Table Partitioning

    Improves performance on large datasets

In [ ]:
CREATE TABLE logs (
  log_id SERIAL,
  log_time TIMESTAMP NOT NULL,
  message TEXT
) PARTITION BY RANGE (log_time);

CREATE TABLE logs_2024 PARTITION OF logs FOR VALUES FROM ('2024-01-01') TO ('2025-01-01');


Use partitioning when handling huge datasets (logs, events, etc.)

---------

## 🚀 Phase 5: Performance & Optimization
- ✅ Indexing

    - `CREATE INDEX, UNIQUE INDEX`

    - Types: 
        - `B-Tree` (default),
        - `GIN` (for JSONB), 
        - `GiST` (for geospatial)

    - Use `EXPLAIN ANALYZE` to measure performance

- ✅ Query Optimization Best Practices - Performance Tuning

    - Avoid `SELECT *` in production

    - Filter using indexed columns

    - Reduce subqueries when unnecessary

    - Use pagination (`LIMIT`, `OFFSET`) smartly

    - Monitor slow queries with `pg_stat_statements`

-----------------------

## 🔁 Phase 6: Functions, Views & Triggers
- ✅ Views

    - CREATE VIEW for virtual tables

    - Use to hide complexity or expose partial data

In [ ]:
CREATE VIEW active_users AS
SELECT id, name FROM users WHERE is_active = TRUE;

- ✅ Triggers

    Auto-run logic on insert/update/delete

In [ ]:
CREATE TRIGGER log_update
AFTER UPDATE ON employees
FOR EACH ROW
EXECUTE FUNCTION log_employee_change();

In [ ]:
CREATE FUNCTION l	IT	90000
og_employee_change() RETURNS trigger AS $$
BEGIN
  INSERT INTO log_table VALUES (OLD.id, now());
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER log_update
AFTER UPDATE ON employees
FOR EACH ROW EXECUTE FUNCTION log_employee_change();

- ✅ Stored Functions (PL/pgSQL)

    Write functions with loops, IF, etc.

In [ ]:
CREATE FUNCTION get_bonus(salary NUMERIC) RETURNS NUMERIC AS $$
BEGIN
  RETURN salary * 0.1;
END;
$$ LANGUAGE plpgsql;

-----------------

## 📦 Phase 7: JSONB, Arrays & Modern Data Types

✅ 1) JSONB (Binary JSON in PostgreSQL)

- Allows you to store and query JSON data efficiently.
- Better performance than `JSON` for querying.

In [ ]:
CREATE TABLE products (
  id SERIAL PRIMARY KEY,
  info JSONB
);

- Query Example:

In [ ]:
SELECT info->>'brand' FROM products WHERE info->>'category' = 'mobile';

- 🔧 JSONB Operators:

    - `->` Get JSON object field by key

    - `->>` Get JSON object field as text

    - `#>>` Get JSON object at path as text

    - `@>` Contains operator

✅ 2) Arrays

- PostgreSQL supports native arrays
- Store multiple values in a single column (homogeneous data types).

In [ ]:
CREATE TABLE devs (
  id SERIAL,
  name TEXT,
  skills TEXT[]
);

SELECT * FROM devs WHERE 'Python' = ANY(skills);

- 💡 Useful Array Functions:

    - unnest(array) – Expands array into a set

    - array_length(array, dimension) – Returns size


✅ 3. UUID (Universally Unique Identifier)

- Use uuid-ossp extension
- Used for generating unique primary keys across distributed systems.

In [ ]:
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";

CREATE TABLE sessions (
  session_id UUID DEFAULT uuid_generate_v4(),
  created_on TIMESTAMP DEFAULT now()
);

✅ 4. BYTEA (Binary Data Type)

- Stores binary files like images, PDFs, audio files.

In [ ]:
CREATE TABLE assets (
  id SERIAL,
  file_name TEXT,
  data BYTEA
);

✅ 5. Real-world Example: Musical CSV 🎵

Use-case: Store musical sequences, notes, rhythm data in arrays or JSONB.

💾 Store in PostgreSQL (JSONB-based):

In [ ]:
CREATE TABLE musical_data (
  id SERIAL PRIMARY KEY,
  track_id INT,
  sequence JSONB
);

🎯 Example Row:

In [ ]:
{
  "notes": [
    {"note": "C4", "duration": 0.5, "volume": 80},
    {"note": "E4", "duration": 0.5, "volume": 90}
  ]
}

-----------------------

## 🔐 Phase 8: Roles, Security & Backup
- ✅ Role Management & Access Control

    - Create users with permissions

    - Use `GRANT`, `REVOKE`, schema-level access

In [ ]:
CREATE ROLE readonly;
GRANT CONNECT ON DATABASE dev_db TO readonly;
GRANT USAGE ON SCHEMA public TO readonly;
GRANT SELECT ON ALL TABLES IN SCHEMA public TO readonly;

- ✅ Backup & Restore

    - pg_dump for logical backups

    - pg_restore for restoring dumps

In [ ]:
pg_dump -U hemendra dev_db > dev_db.sql

psql -U hemendra -d new_db < dev_db.sql

-------------